# Modelo SVC para detección de noticias falsas

En este notebook, entrenaremos un modelo de clasificación basado en **Support Vector Classification (SVC)** para predecir si una afirmación política es **verdadera** o **falsa**.

Se utilizarán técnicas de preprocesamiento de texto (TF-IDF) y metadatos categóricos (codificados y escalados). Además, se aplicará balanceo de clases con SMOTE, selección de características, y búsqueda de hiperparámetros con GridSearchCV.


In [1]:
%pip install pandas numpy matplotlib seaborn
%pip install -U scikit-learn
%pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif

### Carga dataset

Se cargan los datasets de entrenamiento y prueba preprocesados, que contienen:

- `statement`: texto de la afirmación (columna principal para NLP).
- `subject`: tema de la afirmación (categórica).
- `speaker`: persona que hizo la afirmación (categórica).
- `party_affiliation`: partido político (categórica).
- `state_info_*`: columnas relacionadas con la ubicación geográfica.
- `label`: etiqueta binaria (0 = falso, 1 = verdadero) para entrenamiento.

In [3]:
# Cargar los datasets limpios
train_data = pd.read_csv('C:/Users/inesg/dev/LBBYs_CH2/data/processed/train_simp_preprocess_v2.csv')
test_data = pd.read_csv('C:/Users/inesg/dev/LBBYs_CH2/data/processed/test_simp_preprocess_v2.csv')

In [4]:
train_data.head()

,id,label,statement,subject,speaker,speaker_job,state_info,party_affiliation,party_affiliation_uni,party_affiliation_category_map,...,pos_info_without_stopwords,pos_freq_without_stopwords,lemma_freq_without_stopwords,tag_freq_without_stopwords,processed_subject,speaker_entities,speaker_type,speaker_job_tokens,state_info_tokens,party_affiliation_tokens
0,81f884c64a7,1,china is in the south china sea and (building)...,other,donald-trump,president-elect,new york,republican,republican,political-affiliation,...,"[{'lemma': 'china', 'pos': 'PROPN', 'tag': 'NN...","Counter({'PROPN': 4, 'NOUN': 4, 'ADJ': 1, 'VER...","Counter({'china': 2, 'south': 1, 'sea': 1, 'bu...","Counter({'NNP': 4, 'NN': 3, 'JJ': 1, 'NNS': 1,...",[],['donald trump'],['PERSON'],"['president', '-', 'elect']","['new', 'york']",['republican']
1,30c2723a188,0,with the resources it takes to execute just ov...,health-care,chris-dodd,u.s. senator,connecticut,democrat,democrat,political-affiliation,...,"[{'lemma': 'resource', 'pos': 'NOUN', 'tag': '...","Counter({'NOUN': 7, 'VERB': 4, 'PROPN': 2, 'AD...","Counter({'resource': 1, 'take': 1, 'execute': ...","Counter({'NN': 4, 'NNS': 3, 'VB': 2, 'NNP': 2,...",['health-care'],['chris dodd'],['PERSON'],"['u.s', '.', 'senator']",['connecticut'],['democrat']
2,6936b216e5d,0,the (wisconsin) governor has proposed tax give...,other,donna-brazile,other,"washington, d.c.",democrat,democrat,political-affiliation,...,"[{'lemma': 'wisconsin', 'pos': 'PROPN', 'tag':...","Counter({'NOUN': 4, 'PROPN': 1, 'VERB': 1})","Counter({'wisconsin': 1, 'governor': 1, 'propo...","Counter({'NN': 2, 'NNS': 2, 'NNP': 1, 'VBN': 1})",[],['donna brazile'],['PERSON'],['other'],"['washington', ',', 'd.c', '.']",['democrat']
3,b5cd9195738,1,says her representation of an ex-boyfriend who...,other,rebecca-bradley,non-define,non-define,none,none,other-political-groups,...,"[{'lemma': 'say', 'pos': 'VERB', 'tag': 'VBZ',...","Counter({'NOUN': 9, 'VERB': 1, 'ADJ': 1})","Counter({'say': 1, 'representation': 1, 'ex': ...","Counter({'NN': 8, 'VBZ': 1, 'NNS': 1, 'JJ': 1})",[],['rebecca bradley'],['PERSON'],"['non', '-', 'define']","['non', '-', 'define']",['none']
4,84f8dac7737,0,at protests in wisconsin against proposed coll...,other,republican-party-wisconsin,non-define,wisconsin,republican,republican,political-affiliation,...,"[{'lemma': 'protest', 'pos': 'NOUN', 'tag': 'N...","Counter({'NOUN': 7, 'VERB': 4, 'ADJ': 3})","Counter({'protest': 1, 'wisconsin': 1, 'propos...","Counter({'NNS': 4, 'NN': 3, 'JJ': 3, 'VBN': 2,...",[],"['republican party', 'wisconsin']","['ORG', 'GPE']","['non', '-', 'define']",['wisconsin'],['republican']


In [5]:
test_data.head()

,id,statement,subject,speaker,speaker_job,state_info,party_affiliation,party_affiliation_uni,party_affiliation_category_map,statement_tokens,...,pos_info_without_stopwords,pos_freq_without_stopwords,lemma_freq_without_stopwords,tag_freq_without_stopwords,processed_subject,speaker_entities,speaker_type,speaker_job_tokens,state_info_tokens,party_affiliation_tokens
0,dc32e5ffa8b,five members of [the common cause georgia] boa...,other,kasim-reed,non-define,non-define,democrat,democrat,political-affiliation,five members of [ the common cause georgia ] b...,...,"[{'lemma': 'member', 'pos': 'NOUN', 'tag': 'NN...","Counter({'NOUN': 5, 'ADJ': 2, 'PROPN': 1, 'VER...","Counter({'member': 1, 'common': 1, 'cause': 1,...","Counter({'NN': 3, 'NNS': 2, 'JJ': 2, 'NNP': 1,...",[],['PERSON'],[],"['non', '-', 'define']","['non', '-', 'define']",['democrat']
1,aa49bb41cab,theres no negative advertising in my campaign ...,elections,bill-mccollum,non-define,florida,republican,republican,political-affiliation,there s no negative advertising in my campaign...,...,"[{'lemma': 's', 'pos': 'VERB', 'tag': 'VBZ', '...","Counter({'NOUN': 2, 'VERB': 1, 'ADJ': 1})","Counter({'s': 1, 'negative': 1, 'advertising':...","Counter({'NN': 2, 'VBZ': 1, 'JJ': 1})",['elections'],['bill mccollum'],['PERSON'],"['non', '-', 'define']",['florida'],['republican']
2,dddc8d12ac1,leticia van de putte voted to give illegal imm...,other,dan-patrick,other,texas,republican,republican,political-affiliation,leticia van de putte voted to give illegal imm...,...,"[{'lemma': 'leticia', 'pos': 'PROPN', 'tag': '...","Counter({'NOUN': 10, 'ADJ': 4, 'PROPN': 2, 'X'...","Counter({'health': 3, 'care': 3, 'free': 2, 'l...","Counter({'NN': 9, 'JJ': 4, 'NNP': 2, 'FW': 1, ...",[],['dan patrick'],['PERSON'],['other'],['texas'],['republican']
3,bcfe8f51667,fiorinas plan would mean slashing social secur...,other,barbara-boxer,u.s. senator,california,democrat,democrat,political-affiliation,fiorinas plan would mean slashing social secur...,...,"[{'lemma': 'fiorina', 'pos': 'NOUN', 'tag': 'N...","Counter({'NOUN': 4, 'VERB': 2, 'ADJ': 1})","Counter({'fiorina': 1, 'plan': 1, 'mean': 1, '...","Counter({'NN': 3, 'NNS': 1, 'VB': 1, 'VBG': 1,...",[],['barbara boxer'],['PERSON'],"['u.s', '.', 'senator']",['california'],['democrat']
4,eedbbaff5ab,"by the end of his first term, president obama ...",other,mitt-romney,former governor,massachusetts,republican,republican,political-affiliation,"by the end of his first term , president obama...",...,"[{'lemma': 'end', 'pos': 'NOUN', 'tag': 'NN', ...","Counter({'NOUN': 4, 'PROPN': 2, 'VERB': 2, 'AD...","Counter({'president': 2, 'end': 1, 'term': 1, ...","Counter({'NN': 3, 'NNP': 2, 'VBN': 2, 'JJ': 1,...",[],['mitt romney'],['PERSON'],"['former', 'governor']",['massachusetts'],['republican']


### Constantes

In [6]:
submissions_folder = "C:/Users/inesg/dev/LBBYs_CH2/notebooks/3_summision"


### Preprocesamiento de los datos
#### 1. Preprocesamiento del texto con TF-IDF
Se transforma la columna `statement` con TF-IDF, limitando a las 1000 palabras más importantes y eliminando stopwords en inglés.

Esto convierte el texto en vectores numéricos que el modelo puede procesar.

In [7]:
# Crear y entrenar el tokenizer con TF-IDF
tokenizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_text = tokenizer.fit_transform(train_data['statement']).toarray()

#### 2. Preprocesamiento de metadatos
Se codifican las variables categóricas (`subject`, `speaker`, `party_affiliation`) con LabelEncoder y se reduce la información de `state_info_*` a un indicador binario de presencia.

In [8]:
label_encoder_subject = LabelEncoder()
label_encoder_speaker = LabelEncoder()
label_encoder_party = LabelEncoder()

train_data['subject_encoded'] = label_encoder_subject.fit_transform(train_data['subject'])
train_data['speaker_encoded'] = label_encoder_speaker.fit_transform(train_data['speaker'])

state_info_columns = [col for col in train_data.columns if col.startswith('state_info')]
train_data['state_info_encoded'] = train_data[state_info_columns].apply(
    lambda x: 1 if any(isinstance(val, str) and len(val) > 0 for val in x) else 0,
    axis=1
)

train_data['party_affiliation_encoded'] = label_encoder_party.fit_transform(train_data['party_affiliation'])

X_metadata = train_data[['subject_encoded', 'speaker_encoded', 'state_info_encoded', 'party_affiliation_encoded']]

#### 3. Escalar los metadatos
Se normalizan los valores numéricos de los metadatos para que estén en la misma escala y el modelo no se sesgue.

In [9]:
scaler = StandardScaler()
X_metadata_scaled = scaler.fit_transform(X_metadata)

#### 4. Unión texto y metadatos
Se concatenan las características textuales y los metadatos para formar la matriz de características completa.

In [10]:
X_final = np.concatenate([X_text, X_metadata_scaled], axis=1)
y = train_data['label'].values

### División en train/test
Dividimos los datos en conjunto de entrenamiento y prueba para evaluar la generalización.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

### Manejo clases desbalanceadas
Si las clases están desbalanceadas, se aplica `SMOTE` para generar muestras sintéticas de la clase minoritaria solo en el conjunto de entrenamiento.

In [12]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
print(f"Clases balanceadas en train tras SMOTE: {np.bincount(y_resampled)}")

Clases balanceadas en train tras SMOTE: [4616 4616]


### Entrenamiento

#### 1. GridSearchCV
Se busca la mejor combinación de `C`, `gamma` y `kernel` para el modelo SVC, usando validación cruzada y priorizando el recall de la clase 0 (falsas).

(En primer lugar he aplicado el gridSearch priorizando el accuaracy, luego lo he modificado para priorizar el f1-score de la clase 0 que era el  que peor resultados tenia)

In [37]:
from sklearn.metrics import make_scorer, recall_score

In [48]:
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.1, 1],
    'kernel': ['linear', 'rbf']
}

svc = SVC(class_weight='balanced')
# Usar recall de clase 0
scorer_recall_0 = make_scorer(recall_score, pos_label=0)
grid_search = GridSearchCV(svc, param_grid, scoring=scorer_recall_0, cv=5, n_jobs=-1)
grid_search.fit(X_resampled, y_resampled)
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")

best_model = grid_search.best_estimator_

Mejores parámetros encontrados: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}


##### Evaluación

In [49]:
# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Realizar predicciones
y_pred_test = best_model.predict(X_test)

print("Métricas en test:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_test):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_test):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_test):.4f}")
print(classification_report(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))

Métricas en test:
Accuracy: 0.6291
Precision: 0.6799
Recall: 0.8253
F1-Score: 0.7456
              precision    recall  f1-score   support

           0       0.43      0.25      0.32       611
           1       0.68      0.83      0.75      1179

    accuracy                           0.63      1790
   macro avg       0.55      0.54      0.53      1790
weighted avg       0.59      0.63      0.60      1790

[[153 458]
 [206 973]]


##### Submission

In [41]:
def encode_with_unknown_handling(le, series):
    # Valores conocidos en train
    known_labels = set(le.classes_)
    # Reemplazar valores no conocidos por el más frecuente o un valor fijo
    replacement = le.classes_[0]  # O el más frecuente en train
    series_fixed = series.apply(lambda x: x if x in known_labels else replacement)
    return le.transform(series_fixed)

# Aplicar a cada columna categórica
test_data['subject_encoded'] = encode_with_unknown_handling(label_encoder_subject, test_data['subject'])
test_data['speaker_encoded'] = encode_with_unknown_handling(label_encoder_speaker, test_data['speaker'])
test_data['party_affiliation_encoded'] = encode_with_unknown_handling(label_encoder_party, test_data['party_affiliation'])

# Preprocesar texto test
X_test_text = tokenizer.transform(test_data['statement']).toarray()

# Codificar metadatos test
test_data['state_info_encoded'] = test_data[state_info_columns].apply(
    lambda x: 1 if any(isinstance(val, str) and len(val) > 0 for val in x) else 0,
    axis=1
)

X_test_metadata = test_data[['subject_encoded', 'speaker_encoded', 'state_info_encoded', 'party_affiliation_encoded']]

# Escalar metadatos test
X_test_metadata_scaled = scaler.transform(X_test_metadata)

# Concatenar todo para obtener la matriz final con 1004 features
X_test_final = np.concatenate([X_test_text, X_test_metadata_scaled], axis=1)

# Predecir usando el modelo entrenado y matriz final
y_pred_submission = best_model.predict(X_test_final)

# Crear DataFrame submission
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'label': y_pred_submission
})

submission_csv_path = f"{submissions_folder}/submission_grid_search_updated_1.csv"
submission_df.to_csv(submission_csv_path, index=False)

print(f"Submission guardada en: {submission_csv_path}")

Submission guardada en: C:/Users/inesg/dev/LBBYs_CH2/notebooks/3_summision/submission_grid_search_updated_1.csv


#### 2. SelectKBest
Esta técnica selecciona las características más relevantes para la clasificación basándose en el análisis estadístico **ANOVA F-value**.

Seleccionar solo las características más importantes puede ayudar a reducir ruido, mejorar el rendimiento y acelerar el entrenamiento.

In [22]:
selector = SelectKBest(f_classif, k=100)
X_train_selected = selector.fit_transform(X_resampled, y_resampled)
X_test_selected = selector.transform(X_test)


c:\Users\inesg\dev\LBBYs_CH2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [1002] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\inesg\dev\LBBYs_CH2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


##### Evaluación

In [23]:
svc_selected_model = SVC(C=10, gamma=1, kernel='rbf')
svc_selected_model.fit(X_train_selected, y_resampled)

y_pred_selectKBest = svc_selected_model.predict(X_test_selected)

print("Métricas con SelectKBest:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_selectKBest):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selectKBest):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selectKBest):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_selectKBest):.4f}")
print(classification_report(y_test, y_pred_selectKBest))
print(confusion_matrix(y_test, y_pred_selectKBest))

Métricas con SelectKBest:
Accuracy: 0.5642
Precision: 0.7021
Recall: 0.5878
F1-Score: 0.6399
              precision    recall  f1-score   support

           0       0.39      0.52      0.45       611
           1       0.70      0.59      0.64      1179

    accuracy                           0.56      1790
   macro avg       0.55      0.55      0.54      1790
weighted avg       0.60      0.56      0.57      1790

[[317 294]
 [486 693]]


##### Submission

In [24]:
# Preprocesar texto test
X_test_text = tokenizer.transform(test_data['statement']).toarray()

# Codificar metadatos test con manejo de valores desconocidos
def encode_with_unknown_handling(le, series):
    known_labels = set(le.classes_)
    replacement = le.classes_[0]
    series_fixed = series.apply(lambda x: x if x in known_labels else replacement)
    return le.transform(series_fixed)

test_data['subject_encoded'] = encode_with_unknown_handling(label_encoder_subject, test_data['subject'])
test_data['speaker_encoded'] = encode_with_unknown_handling(label_encoder_speaker, test_data['speaker'])

test_data['state_info_encoded'] = test_data[state_info_columns].apply(
    lambda x: 1 if any(isinstance(val, str) and len(val) > 0 for val in x) else 0,
    axis=1
)

test_data['party_affiliation_encoded'] = encode_with_unknown_handling(label_encoder_party, test_data['party_affiliation'])

X_test_metadata = test_data[['subject_encoded', 'speaker_encoded', 'state_info_encoded', 'party_affiliation_encoded']]

# Escalar metadatos test
X_test_metadata_scaled = scaler.transform(X_test_metadata)

# Concatenar texto y metadatos
X_test_final = np.concatenate([X_test_text, X_test_metadata_scaled], axis=1)

# Aplicar SelectKBest al test
X_test_selected = selector.transform(X_test_final)

# Predecir con modelo entrenado
y_pred_selectKBest_submission = svc_selected_model.predict(X_test_selected)

# Crear DataFrame de submission
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'label': y_pred_selectKBest_submission
})

# Guardar CSV
submission_csv_path = f"{submissions_folder}/submission_selectKBest.csv"
submission_df.to_csv(submission_csv_path, index=False)

print(f"Submission guardada en: {submission_csv_path}")


Submission guardada en: C:/Users/inesg/dev/LBBYs_CH2/notebooks/3_summision/submission_selectKBest.csv


#### 3. RFE (Recursive Feature Elimination)

RFE es una técnica que elimina recursivamente las características menos importantes basándose en la importancia que asigna un estimador (en este caso un SVC lineal).

Se entrena un modelo con todas las características, se elimina la menos importante, y se repite hasta quedarse con el número deseado de características.

Esto ayuda a obtener un subconjunto de características muy relevantes para mejorar la generalización y reducir ruido.

In [25]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

In [26]:
svc_base = SVC(kernel='linear', C=1)
rfe_selector = RFE(estimator=svc_base, n_features_to_select=50)

X_train_rfe = rfe_selector.fit_transform(X_resampled, y_resampled)

X_test_rfe = rfe_selector.transform(X_test)


##### Evaluación

In [27]:
svc_rfe_model = SVC(C=10, gamma=1, kernel='rbf')
svc_rfe_model.fit(X_train_rfe, y_resampled)

y_pred_rfe = svc_rfe_model.predict(X_test_rfe)

print(f"Accuracy con RFE: {accuracy_score(y_test, y_pred_rfe):.4f}")
print(f"Precision con RFE: {precision_score(y_test, y_pred_rfe):.4f}")
print(f"Recall con RFE: {recall_score(y_test, y_pred_rfe):.4f}")
print(f"F1-Score con RFE: {f1_score(y_test, y_pred_rfe):.4f}")

print("Reporte de clasificación con RFE:")
print(classification_report(y_test, y_pred_rfe))
print("Matriz de confusión con RFE:")
print(confusion_matrix(y_test, y_pred_rfe))

Accuracy con RFE: 0.5056
Precision con RFE: 0.7466
Recall con RFE: 0.3774
F1-Score con RFE: 0.5014
Reporte de clasificación con RFE:
              precision    recall  f1-score   support

           0       0.39      0.75      0.51       611
           1       0.75      0.38      0.50      1179

    accuracy                           0.51      1790
   macro avg       0.57      0.57      0.51      1790
weighted avg       0.62      0.51      0.50      1790

Matriz de confusión con RFE:
[[460 151]
 [734 445]]


##### Submission

In [28]:
# Preprocesar texto test
X_test_text = tokenizer.transform(test_data['statement']).toarray()

# Codificar metadatos test con manejo de valores desconocidos
def encode_with_unknown_handling(le, series):
    known_labels = set(le.classes_)
    replacement = le.classes_[0]
    series_fixed = series.apply(lambda x: x if x in known_labels else replacement)
    return le.transform(series_fixed)

test_data['subject_encoded'] = encode_with_unknown_handling(label_encoder_subject, test_data['subject'])
test_data['speaker_encoded'] = encode_with_unknown_handling(label_encoder_speaker, test_data['speaker'])

test_data['state_info_encoded'] = test_data[state_info_columns].apply(
    lambda x: 1 if any(isinstance(val, str) and len(val) > 0 for val in x) else 0,
    axis=1
)

test_data['party_affiliation_encoded'] = encode_with_unknown_handling(label_encoder_party, test_data['party_affiliation'])

X_test_metadata = test_data[['subject_encoded', 'speaker_encoded', 'state_info_encoded', 'party_affiliation_encoded']]

# Escalar metadatos test
X_test_metadata_scaled = scaler.transform(X_test_metadata)

# Concatenar texto y metadatos
X_test_final = np.concatenate([X_test_text, X_test_metadata_scaled], axis=1)

# Aplicar RFE selector al test
X_test_rfe = rfe_selector.transform(X_test_final)

# Predecir con modelo entrenado
y_pred_rfe_submission = svc_rfe_model.predict(X_test_rfe)

# Crear DataFrame de submission
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'label': y_pred_rfe_submission
})

# Guardar CSV
submission_csv_path = f"{submissions_folder}/submission_rfe.csv"
submission_df.to_csv(submission_csv_path, index=False)

print(f"Submission guardada en: {submission_csv_path}")


Submission guardada en: C:/Users/inesg/dev/LBBYs_CH2/notebooks/3_summision/submission_rfe.csv


# Conclusiones sobre las técnicas usadas en el modelo SVC para detección de noticias falsas

## 1. GridSearchCV

- GridSearchCV es una técnica que nos ayuda a encontrar la mejor combinación de hiperparámetros para el modelo. En este caso, buscábamos los valores óptimos para parámetros como `C`, `gamma` y el tipo de kernel en el SVC.
- Primero intentamos optimizar el modelo para que tuviera la mejor precisión global (accuracy), pero noté que la clase 0 (noticias falsas) tenía resultados muy malos.
- Entonces, cambié el enfoque para que GridSearch priorizara el f1-score de la clase 0, intentando mejorar la capacidad del modelo para detectar esas noticias falsas.
- Finalmente, GridSearch indicó que la mejor configuración era: `C=10`, `gamma=1` y `kernel='rbf'`.
- Con esos parámetros el modelo mejoró, pero todavía no detectaba bien la clase 0. Esto ocurre porque aunque GridSearch optimiza los parámetros, el modelo y los datos tienen limitaciones propias que no se solucionan solo con ajustar parámetros.

## 2. SelectKBest

- SelectKBest ayudó a reducir la dimensionalidad seleccionando las características más relevantes basadas en pruebas estadísticas.
- Esto contribuyó a eliminar ruido y posibles variables poco informativas, mejorando la eficiencia del entrenamiento y la generalización del modelo.
- Sin embargo, la mejora en métricas no fue muy destacable, lo que sugiere que el conjunto de características ya contenía cierta redundancia o que las características seleccionadas no capturaban suficientemente la complejidad del problema.
- Además, al tratarse de un problema textual y semántico, la selección basada solo en criterios estadísticos simples puede no ser suficiente para identificar las verdaderas variables clave.

## 3. RFE (Recursive Feature Elimination)

- RFE se usó para eliminar recursivamente características menos importantes con el objetivo de mejorar el rendimiento del modelo y evitar sobreajuste.
- Esta técnica permitió identificar un subconjunto más pequeño de variables, simplificando el modelo y potencialmente mejorando su interpretabilidad.
- A pesar de esto, las métricas, especialmente para la clase minoritaria, no mejoraron significativamente, evidenciando que el problema principal no es solo el exceso de características, sino la dificultad del dataset y la representación de la información.
- RFE con un modelo SVC lineal puede ser demasiado restrictivo para un problema tan complejo como la detección de noticias falsas, donde las relaciones entre variables pueden ser no lineales y de alta dimensión.

---

### Resumen general

Cada técnica contribuyó de alguna forma a mejorar el modelo o su interpretación, pero ninguna por sí sola logró resolver los principales desafíos del problema:

- GridSearchCV afinó hiperparámetros pero no pudo compensar las limitaciones del modelo y los datos.  
- SelectKBest y RFE ayudaron a reducir características irrelevantes, pero no lograron mejorar la detección de la clase minoritaria.  

Esto indica que para mejorar los resultados sería necesario combinar estas técnicas con:

- Representaciones más avanzadas del texto.  
- Modelos más potentes y flexibles que puedan capturar patrones complejos.  
- Métodos más robustos para manejar el desbalance y la ambigüedad en las etiquetas.
